In [3]:
#Dependencies
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [4]:
#read csv into df
df = pd.read_csv("https://data-finalproject.s3.amazonaws.com/Levels_Fyi_Salary_Data.csv")
# columns to drop... salary_currency, salary, work_year
df.sample(25)


,timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,...,Doctorate_Degree,Highschool,Some_College,Race_Asian,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic,Race,Education
1933,10/13/2018 4:55:55,Uber,L4,Software Engineer,308000,"San Francisco, CA",2.0,0.0,Distributed Systems (Back-End),155000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
30323,9/18/2020 18:56:21,Autodesk,Grade 12,Software Engineer,115000,"Singapore, SG, Singapore",6.0,4.0,API Development (Back-End),101000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
28490,9/2/2020 8:28:47,Docusign,L4,Marketing,416000,"San Francisco, CA",10.0,4.0,Product,178000.0,...,0,0,0,1,0,0,0,0,Asian,Master's Degree
46521,3/30/2021 6:08:04,Activision,NaN,Product Manager,161000,"Santa Monica, CA",4.0,0.0,Product,140000.0,...,0,0,0,0,1,0,0,0,White,Bachelor's Degree
62367,2/4/2018 15:46:05,Amazon,SDE I,Software Engineer,157750,"Seattle, WA",0.0,0.0,NaN,106000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
49941,4/28/2021 23:54:56,Dropbox,IC2,Software Engineer,268000,"San Francisco, CA",2.0,0.0,Web Development (Front-End),180000.0,...,0,0,0,0,0,0,1,0,Black,Master's Degree
12604,11/1/2019 9:42:23,Nvidia,IC4,Software Engineer,235000,"Santa Clara, CA",3.0,0.5,ML / AI,185000.0,...,1,0,0,0,0,0,0,0,NaN,PhD
34914,11/9/2020 23:34:11,Oracle,IC-1,Software Engineer,12000,"Bangalore, KA, India",1.0,1.0,Networking,12000.0,...,0,0,0,1,0,0,0,0,Asian,Bachelor's Degree
52076,5/22/2021 5:09:43,Amazon,L5,Software Engineer,70000,"Gurgaon, HR, India",5.0,3.0,Distributed Systems (Back-End),35000.0,...,0,0,0,1,0,0,0,0,Asian,Bachelor's Degree
14397,12/28/2019 20:51:41,Facebook,L6 Product Manager,Product Manager,460000,"Menlo Park, CA",15.0,0.0,Product management,225000.0,...,0,0,0,0,0,0,0,0,NaN,Master's Degree


In [5]:
df.nunique()


timestamp                  62561
company                     1631
level                       2923
title                         15
totalyearlycompensation      893
location                    1050
yearsofexperience             65
yearsatcompany                81
tag                         3058
basesalary                   482
stockgrantvalue              612
bonus                        335
gender                         4
otherdetails               12841
cityid                      1045
dmaid                        149
rowNumber                  62642
Masters_Degree                 2
Bachelors_Degree               2
Doctorate_Degree               2
Highschool                     2
Some_College                   2
Race_Asian                     2
Race_White                     2
Race_Two_Or_More               2
Race_Black                     2
Race_Hispanic                  2
Race                           5
Education                      5
dtype: int64

In [6]:
titles_df = df[df["title"]=="Data Scientist"]
titles_df


,timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,...,Doctorate_Degree,Highschool,Some_College,Race_Asian,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic,Race,Education
419,6/5/2018 14:06:30,LinkedIn,Senior,Data Scientist,233000,"San Francisco, CA",4.0,0.0,Data Analysis,162000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
440,6/8/2018 9:49:25,Microsoft,64,Data Scientist,218000,"Seattle, WA",11.0,11.0,ML / AI,165000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
444,6/8/2018 17:55:09,ebay,26,Data Scientist,180000,"San Jose, CA",10.0,5.0,NaN,0.0,...,0,0,0,0,0,0,0,0,NaN,NaN
454,6/10/2018 19:39:35,Twitter,Staff,Data Scientist,500000,"San Francisco, CA",4.0,4.0,ML / AI,200000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
495,6/17/2018 11:39:38,Facebook,5,Data Scientist,370000,"Seattle, WA",8.0,3.0,NaN,190000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62240,3/26/2019 17:25:02,IBM,8,Data Scientist,155000,"Washington, DC",5.0,2.0,ML / AI,141000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
62283,6/17/2018 21:07:18,Facebook,E3,Data Scientist,150000,"Singapore, SG, Singapore",0.0,0.0,ML / AI,150000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
62285,7/16/2018 20:11:40,ADP,Sr UX Researcher 4,Data Scientist,185000,"Roseland, NJ",8.0,2.0,UX Research,150000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
62529,6/12/2018 20:54:06,Google,T6,Data Scientist,685000,"Kirkland, WA",22.0,2.0,ML / AI,221000.0,...,0,0,0,0,0,0,0,0,NaN,NaN


In [7]:
# df1 = titles_df.copy()
# filtered_columns = ["company", "location", "yearsofexperience", "totalyearlycompensation",
#                  "Masters_Degree", "Bachelors_Degree"]
# df2 = df1[filtered_columns]
# # print(df1.shape)
# df2.head()

In [8]:
clean_df= titles_df.drop(columns=['timestamp', 'level', 'yearsatcompany', 'otherdetails','Highschool', 'Some_College', 
                               'Race_Asian', 'Race_White', 'Race_Two_Or_More', 'Race_Black',"gender",
                              'Race_Hispanic', 'Race', 'Education','cityid','dmaid','rowNumber','tag','bonus','stockgrantvalue','basesalary','title'])
clean_df.head()
#get_dummies/oneHotEncoder on company size and employment type

,company,totalyearlycompensation,location,yearsofexperience,Masters_Degree,Bachelors_Degree,Doctorate_Degree
419,LinkedIn,233000,"San Francisco, CA",4.0,0,0,0
440,Microsoft,218000,"Seattle, WA",11.0,0,0,0
444,ebay,180000,"San Jose, CA",10.0,0,0,0
454,Twitter,500000,"San Francisco, CA",4.0,0,0,0
495,Facebook,370000,"Seattle, WA",8.0,0,0,0


In [9]:
import re
form_state_one = r'(\w+\s)*(\w+),\s([A-Z]{2})$'
matches_form_one = clean_df.location.str.contains(form_state_one, flags=re.IGNORECASE, na=False)
no_form_one = clean_df.location.str.contains(form_state_one, flags=re.IGNORECASE, na=False).sum()
print(no_form_one)

form_state_two = r'(\w+\s)*(\w+),\s([A-Z]{2}),\s(\w+)'
matches_form_two = clean_df.location.str.contains(form_state_two, flags=re.IGNORECASE, na=False)
no_form_two = clean_df.location.str.contains(form_state_two, flags=re.IGNORECASE, na=False).sum()
print(no_form_two)


clean_df.location[~matches_form_one & ~matches_form_two]

2226
351


C:\Users\siriu\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\siriu\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:4: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  after removing the cwd from sys.path.
C:\Users\siriu\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:8: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  
C:\Users\siriu\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:9: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  if __name__ == '__main__':


9061    Tel Aviv, Israel
Name: location, dtype: object

In [10]:
clean_df.head()

,company,totalyearlycompensation,location,yearsofexperience,Masters_Degree,Bachelors_Degree,Doctorate_Degree
419,LinkedIn,233000,"San Francisco, CA",4.0,0,0,0
440,Microsoft,218000,"Seattle, WA",11.0,0,0,0
444,ebay,180000,"San Jose, CA",10.0,0,0,0
454,Twitter,500000,"San Francisco, CA",4.0,0,0,0
495,Facebook,370000,"Seattle, WA",8.0,0,0,0


In [11]:
def US_location_to_state(city):
    # if input is of the form $###,###,###
    if re.match(form_state_one, city, flags=re.IGNORECASE):

        # remove city name and commas
        state = re.sub('(\w+\s)*(\w+),\s','', city)
        
        # return state
        return state
    # otherwise, return NaN
    else:
        return np.nan
    

def location_to_city(city):
    # if input is of the form $###,###,###
    if re.match(form_state_one, city, flags=re.IGNORECASE):

        # remove city name and commas
        city = re.sub(',\s([A-Z]{2})$','', city)
        
        # return state
        return city
    # otherwise, return NaN
    else:
        return np.nan
clean_df = clean_df.loc[clean_df.location != "Tel Aviv, Israel"]
clean_df


,company,totalyearlycompensation,location,yearsofexperience,Masters_Degree,Bachelors_Degree,Doctorate_Degree
419,LinkedIn,233000,"San Francisco, CA",4.0,0,0,0
440,Microsoft,218000,"Seattle, WA",11.0,0,0,0
444,ebay,180000,"San Jose, CA",10.0,0,0,0
454,Twitter,500000,"San Francisco, CA",4.0,0,0,0
495,Facebook,370000,"Seattle, WA",8.0,0,0,0
...,...,...,...,...,...,...,...
62240,IBM,155000,"Washington, DC",5.0,0,0,0
62283,Facebook,150000,"Singapore, SG, Singapore",0.0,0,0,0
62285,ADP,185000,"Roseland, NJ",8.0,0,0,0
62529,Google,685000,"Kirkland, WA",22.0,0,0,0


In [12]:
clean_df['state'] = clean_df.location.str.extract(f'({form_state_one}|{form_state_two})', flags=re.IGNORECASE)[0].apply(US_location_to_state)

In [13]:
clean_df = clean_df[clean_df['state'].notna()]
clean_df.head()

,company,totalyearlycompensation,location,yearsofexperience,Masters_Degree,Bachelors_Degree,Doctorate_Degree,state
419,LinkedIn,233000,"San Francisco, CA",4.0,0,0,0,CA
440,Microsoft,218000,"Seattle, WA",11.0,0,0,0,WA
444,ebay,180000,"San Jose, CA",10.0,0,0,0,CA
454,Twitter,500000,"San Francisco, CA",4.0,0,0,0,CA
495,Facebook,370000,"Seattle, WA",8.0,0,0,0,WA


In [14]:
clean_df['city'] = clean_df.location.str.extract(f'({form_state_one})', flags=re.IGNORECASE)[0].apply(location_to_city)
clean_df.drop('location', axis=1, inplace=True)
clean_df.head()

C:\Users\siriu\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\siriu\anaconda3\envs\mlenv\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,company,totalyearlycompensation,yearsofexperience,Masters_Degree,Bachelors_Degree,Doctorate_Degree,state,city
419,LinkedIn,233000,4.0,0,0,0,CA,San Francisco
440,Microsoft,218000,11.0,0,0,0,WA,Seattle
444,ebay,180000,10.0,0,0,0,CA,San Jose
454,Twitter,500000,4.0,0,0,0,CA,San Francisco
495,Facebook,370000,8.0,0,0,0,WA,Seattle


In [15]:
print(clean_df["company"].value_counts().head(10))

Amazon          224
Microsoft       202
Facebook        175
Google           83
Apple            75
Uber             60
IBM              58
Capital One      44
LinkedIn         34
Walmart Labs     33
Name: company, dtype: int64


In [16]:
company_counts = clean_df['company'].value_counts()
company_counts.head(10)

Amazon          224
Microsoft       202
Facebook        175
Google           83
Apple            75
Uber             60
IBM              58
Capital One      44
LinkedIn         34
Walmart Labs     33
Name: company, dtype: int64

In [17]:
replace_company = list(company_counts[company_counts < 30].index)

# Replace in dataframe
for app in replace_company:
    clean_df.company =  clean_df.company.replace(app,"Other")
    
# Check to make sure binning was successful
clean_df.company.value_counts()

C:\Users\siriu\anaconda3\envs\mlenv\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


Other             1177
Amazon             224
Microsoft          202
Facebook           175
Google              83
Apple               75
Uber                60
IBM                 58
Capital One         44
LinkedIn            34
Walmart Labs        33
JPMorgan Chase      31
Netflix             30
Name: company, dtype: int64

In [37]:
# #Binning he companies- .cut with 4 bins...do we want to specifically design these?
# bin_labels = ['Small', 'Medium', 'Large', 'Very Large']
# bins4 = clean_df['company'].value_counts()
# company_bins = pd.cut(bins4, bins=4, labels=bin_labels).to_list()#does this solve the issue of imbalanced data?
# company_bins


['Very Large',
 'Very Large',
 'Very Large',
 'Medium',
 'Medium',
 'Medium',
 'Medium',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 

In [38]:
# #change company name to bin name
# companies = []
# for company in clean_df['company']:
#     if company == "Amazon" or company == "Microsoft" or company == "Facebook":
#         company = "Large"
#     elif company == "Google" or company == "Apple" or company == "IBM" or company=="Uber":
#         company = "Medium"
#     else:
#         company = "Small"
#     companies.append(company)

In [19]:
# #reassign company column to bin name
# clean_bins_df = clean_df.copy()
# clean_bins_df['company'] = companies
# clean_bins_df.head(10)
# # clean_bins_df['gender'].value_counts()

NameError: name 'companies' is not defined

In [26]:
location_counts = clean_df['state'].value_counts()
location_counts

CA    960
WA    450
NY    254
TX     81
MA     70
IL     48
DC     42
VA     41
NC     33
OR     33
GA     31
CO     23
PA     19
NJ     18
AR     14
OH     14
AZ     13
MI     12
FL     10
MN      9
CT      7
DE      5
UT      4
MD      4
MO      4
IN      4
AL      3
TN      3
ID      3
RI      2
WI      2
NV      2
OK      2
KS      1
KY      1
SC      1
NM      1
MS      1
NH      1
Name: state, dtype: int64

In [28]:
replace_location = list(location_counts[location_counts < 40].index)

# Replace in dataframe
for loc in replace_location:
    clean_df.state =  clean_df.state.replace(loc,"Other")
    
# Check to make sure binning was successful
clean_df.state.value_counts()

CA       960
WA       450
Other    280
NY       254
TX        81
MA        70
IL        48
DC        42
VA        41
Name: state, dtype: int64

In [29]:
clean_dummies = pd.get_dummies(data=clean_bins_df, columns=['company', 'state', 'city'], drop_first=True)#drop the gender columns?
clean_dummies.head(15)

,totalyearlycompensation,yearsofexperience,Masters_Degree,Bachelors_Degree,Doctorate_Degree,company_Apple,company_Capital One,company_Facebook,company_Google,company_IBM,...,city_Walnut Creek,city_Waltham,city_Warren,city_Washington,city_Watertown,city_Wellesley,city_West McLean,city_Wilmington,city_Woonsocket,city_Worcester
419,233000,4.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
440,218000,11.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
444,180000,10.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
454,500000,4.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
495,370000,8.0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
499,200000,3.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
509,340000,11.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
510,690000,10.0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
513,600000,3.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
523,168000,8.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
clean_dummies.to_csv("Cleaned_dataset.csv", sep=',', header=True, encoding='utf-8', index=False)

In [19]:
# #is there linearity between location numerical representation and salary?
# clean_bins2 = clean_bins_df.drop(['location'], axis = 1)
# locations = clean_bins_df['location']
# clean_bins2.head()

In [ ]:
# #Create label Encoder and encode categorical
# from sklearn.preprocessing import LabelEncoder
# lb_make = LabelEncoder()
# clean_bins2['company']= lb_make.fit_transform(clean_bins2['company'])
# clean_bins2['gender']= lb_make.fit_transform(clean_bins2['gender'])

# clean_bins2.head()

In [ ]:
# clean_bins2['gender'].value_counts() #what do we do with 4 genders?